In [1]:
# Importing used packages and model
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
model = read_sbml_model('../src/models/gen/iMT1026-v3.1.xml')
%run -i create_pathway.ipynb  # runs other script with included function to alter model

In [3]:
# Realistic methanol uptake rates cf.Tomàs‐Gamisans 2018
create_pathway(model,1)

medium = model.medium
medium['Ex_meoh'] = 6
medium['Ex_glyc'] = 0
model.medium = medium
model.objective = model.reactions.Ex_biomass
solution = model.optimize()
solution.fluxes['Ex_biomass']

Ignoring reaction 'MSADH' since it already exists.
Ignoring reaction 'mln_c-e' since it already exists.
Ignoring reaction 'Ex_mln' since it already exists.


Added beta alanine pathway


0.09511188720061796

In [3]:
# NADH and NADPH fva summaries at 100% growth and the associated escher fluxes
with model:
    
    model.objective = model.reactions.Ex_biomass 
    solution = model.optimize()
    print(model.metabolites.nadh_c.summary(fva=1.0))
    print(model.metabolites.nadph_c.summary(fva=1.0))
    
import escher
from IPython.display import display
b = escher.Builder(map_json = '../misc/iMM904.Central_carbon_metabolism.json', reaction_data = solution.fluxes.to_dict(),highlight_missing=True, model=model)
display(b)
    

Ignoring reaction 'MSADH' since it already exists.
Ignoring reaction 'mln_c-e' since it already exists.
Ignoring reaction 'Ex_mln' since it already exists.


Added beta alanine pathway
nadh_c
Formula: C21H27N7O14P2

Producing Reactions
-------------------
Percent      Flux                 Range    Reaction                                                     Definition
  0.00%         0          [0; 0.02774]    2OXOADOX       2oxoadp_c + coa_c + nad_c --> co2_c + glutcoa_c + nadh_c
  0.00%         0        [0; 3.444E-05]      AASAD3    L2aadp6sa_c + h2o_c + nad_c --> L2aadp_c + 2.0 h_c + nadh_c
  0.00%         0        [0; 0.0001378]      ALCD19                     glyc_c + nad_c <=> glyald_c + h_c + nadh_c
  0.00%         0   [-4.305E-06; 995.1]      ALCD2x                      etoh_c + nad_c <=> acald_c + h_c + nadh_c
  0.00%         0        [0; 5.103E-06]     ALDD22x         aproa_c + h2o_c + nad_c --> ala_B_c + 2.0 h_c + nadh_c
  0.00%         0        [0; 6.888E-05]      ALDD2x            acald_c + h2o_c + nad_c --> ac_c + 2.0 h_c + nadh_c
  0.00%         0        [0; 2.698E-06]      AM6SAD         am6sa_c + h2o_c + nad_c --> amuco_c +

Builder(highlight_missing=True, reaction_data={'ADPtn': 0.0, 'ATPtn': 0.0, 'FACOAE140': 0.0, 'FACOAL260': 0.0,…

FALDH2 and FDH each produces 41.1% of NADH under pure growth. What about when also producing malonate?

In [9]:
# NADH and NADPH fva summaries at constrained growth (50%) while maximizing for malonate prodcution 
# and the associated escher fluxes
with model:
    
    model.objective = model.reactions.Ex_biomass 
    specified_growth_rate = 0.5*model.optimize().objective_value


    constrained_growth = model.problem.Constraint(
        model.reactions.get_by_id('Ex_biomass').flux_expression,
        lb=specified_growth_rate,
        ub=specified_growth_rate)
    model.add_cons_vars(constrained_growth)
    
    model.objective = model.reactions.Ex_mln
    solution = model.optimize()
    print(model.metabolites.nadh_c.summary(fva=1.0))
    print(model.metabolites.nadph_c.summary(fva=1.0))
b = escher.Builder(map_json = 'iMM904.Central_carbon_metabolism.json', reaction_data = solution.fluxes.to_dict(),highlight_missing=True, model=model)
display(b)
    

Ignoring reaction 'MSADH' since it already exists.
Ignoring reaction 'mln_c-e' since it already exists.
Ignoring reaction 'Ex_mln' since it already exists.


Added beta alanine pathway
nadh_c
Formula: C21H27N7O14P2

Producing Reactions
-------------------
Percent      Flux                   Range    Reaction                                                     Definition
  0.00%         0            [0; 0.01386]    2OXOADOX       2oxoadp_c + coa_c + nad_c --> co2_c + glutcoa_c + nadh_c
  0.00%         0          [0; 1.309E-05]      AASAD3    L2aadp6sa_c + h2o_c + nad_c --> L2aadp_c + 2.0 h_c + nadh_c
  0.00%         0          [0; 5.236E-05]      ALCD19                     glyc_c + nad_c <=> glyald_c + h_c + nadh_c
  0.00%         0     [-1.636E-06; 996.1]      ALCD2x                      etoh_c + nad_c <=> acald_c + h_c + nadh_c
  0.00%         0          [0; 1.939E-06]     ALDD22x         aproa_c + h2o_c + nad_c --> ala_B_c + 2.0 h_c + nadh_c
  0.00%         0          [0; 2.618E-05]      ALDD2x            acald_c + h2o_c + nad_c --> ac_c + 2.0 h_c + nadh_c
  0.00%         0          [0; 1.027E-06]      AM6SAD         am6sa_c + h2o_c + nad

Builder(highlight_missing=True, reaction_data={'ADPtn': 0.0, 'ATPtn': 0.0, 'FACOAE140': 0.0, 'FACOAL260': 0.0,…

Reduction to 15.2% for FALDH2 and FDH. Instead 29.2% comes from MSADH, the malonate producing reaction. Calculating yield of malonate.

In [12]:
mln_molar_mass = 104.06 #g/mol
meoh_molar_mass = 32.04 #g/mol
print('Yield of malonate is', solution.fluxes['Ex_mln']/(-solution.fluxes['Ex_meoh'])*mln_molar_mass/meoh_molar_mass,'g mln/g meoh')


Yield of malonate is 0.6200169492994864 g mln/g meoh


In [13]:
# NADH and NADPH fva summaries at constrained growth (50%) while maximizing for beta-alinine prodcution 
# and the associated escher fluxes
with model:
    Beta_alanine_exchange = Reaction('Ex_ala_B')
    Beta_alanine_exchange.add_metabolites({model.metabolites.ala_B_c: -1})
    model.add_reaction(Beta_alanine_exchange)
    model.reactions.Ex_ala_B.lower_bound = 0
    
    model.objective = model.reactions.Ex_biomass 
    specified_growth_rate = 0.5*model.optimize().objective_value


    constrained_growth = model.problem.Constraint(
        model.reactions.get_by_id('Ex_biomass').flux_expression,
        lb=specified_growth_rate,
        ub=specified_growth_rate)
    model.add_cons_vars(constrained_growth)
    
    model.objective = model.reactions.Ex_ala_B
    solution = model.optimize()
    print(model.metabolites.nadh_c.summary(fva=1.0))
    print(model.metabolites.nadph_c.summary(fva=1.0))
b = escher.Builder(map_json = '../misc/iMM904.Central_carbon_metabolism.json', reaction_data = solution.fluxes.to_dict(),highlight_missing=True, model=model)
display(b)    
    

nadh_c
Formula: C21H27N7O14P2

Producing Reactions
-------------------
Percent      Flux                   Range    Reaction                                                     Definition
  0.00%         0            [0; 0.01384]    2OXOADOX       2oxoadp_c + coa_c + nad_c --> co2_c + glutcoa_c + nadh_c
  0.00%         0          [0; 1.266E-06]      AASAD3    L2aadp6sa_c + h2o_c + nad_c --> L2aadp_c + 2.0 h_c + nadh_c
  0.00%         0          [0; 5.065E-06]      ALCD19                     glyc_c + nad_c <=> glyald_c + h_c + nadh_c
  0.00%         0     [-1.523E-07; 996.3]      ALCD2x                      etoh_c + nad_c <=> acald_c + h_c + nadh_c
  0.00%         0          [0; 1.876E-07]     ALDD22x         aproa_c + h2o_c + nad_c --> ala_B_c + 2.0 h_c + nadh_c
  0.00%         0          [0; 2.532E-06]      ALDD2x            acald_c + h2o_c + nad_c --> ac_c + 2.0 h_c + nadh_c
  0.01% 0.0005599  [0.0005599; 0.0005599]     C3STDH1         4mzym_int1_c + nad_c --> 4mzym_int2_c + co2_c + 

Builder(highlight_missing=True, reaction_data={'ADPtn': 0.0, 'ATPtn': 0.0, 'FACOAE140': 0.0, 'FACOAL260': 0.0,…

Without MSADH as the final NADH producing step, more reducing power is needed from FALDH2 and FDH (33.8% each). The yield then becomes lower.

In [15]:
ala_B_molar_mass = 89.09
print('Yield of beta alanine is', solution.fluxes['Ex_ala_B']/(-solution.fluxes['Ex_meoh'])*ala_B_molar_mass/meoh_molar_mass,'g ala_B/g meoh')

Yield of malonate is 0.3639919396944083 g ala_B/g meoh


***Example of a cofactor swap***

In [5]:
model.objective = model.reactions.Ex_biomass
solution = model.optimize()
solution.fluxes['Ex_biomass']

0.09511188720061796

In [1]:
with model:
    # Trying to cofactor swap NADH to NADPH in GAPD
    cofactor_swap = Reaction('GAPD2') 
    cofactor_swap.add_metabolites({model.metabolites.g3p_c: -1,
                              model.metabolites.nadp_c: -1,
                              model.metabolites.pi_c: -1,
                              model.metabolites.nadph_c: 1,
                              model.metabolites.h_c: 1,
                              model.metabolites.get_by_id('13dpg_c'): 1
                             })
    model.add_reactions([cofactor_swap])
    print(model.reactions.GAPD2)
    
    
    model.objective = model.reactions.Ex_biomass
    solution = model.optimize()
    print('The maximum growth rate is', solution.fluxes['Ex_biomass'])
    print(model.metabolites.nadh_c.summary(fva=1.0))
    print(model.metabolites.nadph_c.summary(fva=1.0))
    

NameError: name 'model' is not defined